# Week 3 PGA
Let's download the libraries first

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 6.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 18.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geocoder

     |████████████████████████████████| 102kB 8.0MB/s eta 0:00:01
     |████████████████████████████████| 829kB 21.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 13.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge


Every library needed is imported

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Get the wiki url

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url = requests.get(url).text

By using the url the entire page code is obtained. Then the the code for the table is saved in "table"

In [7]:
soup = BeautifulSoup(url,"lxml")          #does some magic to get html file
print(soup.prettify())
table = soup.find('table',{'class':'wikitable sortable'})       #gets the code for the wiki table

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xk1RrQpAMFMAAFNtgQ4AAAAJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

Table is filtered according to the data required

In [8]:
rows = table.find_all('tr')
rows

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_Yor

The existence of duplicates is verified by obtaining every row of the original table. It's assumed that text is consistently written for each row.

In [9]:
df = pd.DataFrame(columns = ["Postal Code", "Borough", "Neighborhood"])


for row in rows:           
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    try:                            #just to avoid problems with 1st row which is empty
        if cols[1] != "Not assigned":
            if cols[2] != "Not assigned":
                df = df.append({"Postal Code": cols[0], "Borough": cols[1], "Neighborhood": cols[2]}, ignore_index=True)
            else:
                df = df.append({"Postal Code": cols[0], "Borough": cols[1], "Neighborhood": cols[1]}, ignore_index=True)
    except:
        continue
print(df.shape)
print(df.drop_duplicates().shape) #210 = 210 which is ok (no drop means no duplicates)
df = df.drop(["Neighborhood"], axis=1)
print(df.drop_duplicates().shape) #This should be the total rows for the final dataframe

(210, 3)
(210, 3)
(103, 2)


The dataframe is constructed row by row taking into account each point mentioned in the assignment

In [10]:
df = pd.DataFrame(columns = ["Postal Code", "Borough", "Neighborhood"])   #redefine

#It was verified beforehand that there aren't duplicates by obtaining every row of the original table. It's assumed that text is correctly and consistently written for each row.

for row in rows[1:]:           #1st row has no value
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]      #at this step we got a list with the relevant values for each row, keeping the original order
    if cols[1] != "Not assigned":
        if cols[2] != "Not assigned":             
            try:                                                                               #"try" because if a value is not in the df it returns error
                temp_borough = df["Borough"][df[df["Postal Code"] == cols[0]].index.values[0]]
                if (cols[0] in df["Postal Code"].values) and (temp_borough == cols[1]):        #basically, if the postal code-borough pair already exists in the developing df                       
                    df["Neighborhood"][df["Postal Code"] == cols[0]] = temp_borough + ", " + cols[2]                   
            except:
                 df = df.append({"Postal Code": cols[0], "Borough": cols[1], "Neighborhood": cols[2]}, ignore_index=True)
                    
        else:
            df = df.append({"Postal Code": cols[0], "Borough": cols[1], "Neighborhood": cols[1]}, ignore_index=True)

df.shape     #103 rows as wanted; no more code required

(103, 3)

We obtain location data from csv because geocode doesn't work (not efficiently at least) with the code given

In [11]:
!wget -q -O 'LongLatLocations.csv' http://cocl.us/Geospatial_data
lldf = pd.read_csv('LongLatLocations.csv')
print(lldf.shape) # 103 rows, coincidence?
lldf.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's see if postal codes from "df" and .csv file match

In [12]:
lldf["Postal Code"].unique().shape #103 means no duplicates
for i in lldf["Postal Code"]:
    if i not in df["Postal Code"].values:
        print("Postal Codes from csv file are not the same as the ones from the dataframe constructed")
        break           #So, every postal code is in "df", which means they are the same

As postal codes match, let's just merge the columns from "lldf" to "df"

In [13]:
df.sort_values(["Postal Code"], inplace = True, axis = 0)  #Being cautious to match every row (1)
lldf.sort_values(["Postal Code"], inplace = True, axis = 0) #Being cautious to match every row (2)
df.reset_index(inplace=True, drop=True) #Being cautious to match every row (3)
df["Latitude"] = lldf["Latitude"]
df["Longitude"] = lldf["Longitude"]
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Scarborough, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Scarborough, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Etobicoke, St. Phillips",43.688905,-79.554724
101,M9V,Etobicoke,"Etobicoke, Thistletown",43.739416,-79.588437


A new dataframe wich only contains Toronto boroughs is created

In [14]:
dft = df.copy(deep = True)

for ind, bor in enumerate(dft["Borough"].values):
    if "Toronto" not in bor:
        dft.drop(ind, axis=0, inplace=True)
dft.reset_index(inplace=True, drop=True)
dft

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"East Toronto, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"East Toronto, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Central Toronto, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Central Toronto, Summerhill West",43.686412,-79.400049


## Explore Neighborhoods in Toronto

From this point, the neighbourhoods lab is followed

#### Define Foursquare Credentials and Version

We get the info to use Foursquare API and define a couple functions

In [38]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '******' # your Foursquare Secret
VERSION = '20200218'

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get nearby venues for every Toronto borough

In [19]:
toronto_venues = getNearbyVenues(names=dft['Neighborhood'],
                                   latitudes=dft['Latitude'],
                                   longitudes=dft['Longitude']
                                  )

The Beaches
East Toronto, Riverdale
East Toronto, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Central Toronto, Summerhill East
Central Toronto, Summerhill West
Rosedale
Downtown Toronto, St. James Town
Church and Wellesley
Harbourfront
Downtown Toronto, Garden District
St. James Town
Berczy Park
Central Bay Street
Downtown Toronto, Richmond
Downtown Toronto, Union Station
Downtown Toronto, Toronto Dominion Centre
Downtown Toronto, Victoria Hotel
Roselawn
Central Toronto, Forest Hill West
Central Toronto, Yorkville
Downtown Toronto, University of Toronto
Downtown Toronto, Kensington Market
Downtown Toronto, South Niagara
Stn A PO Boxes 25 The Esplanade
Downtown Toronto, Underground city
Christie
West Toronto, Dufferin
West Toronto, Trinity
West Toronto, Parkdale Village
West Toronto, The Junction South
West Toronto, Roncesvalles
West Toronto, Swansea
Queen's Park
Business Reply Mail Processing Centre 969 Eastern


## Analyze Each Neighborhood

The 10 most common venues for each borough are retrieved by processing the data as dummy variables

In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00
1,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Central Bay Street,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,...,0.00000,0.000000,0.000000,0.00,0.012048,0.000000,0.000000,0.012048,0.000000,0.00
3,"Central Toronto, Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"Central Toronto, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Central Toronto, Summerhill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.00
6,"Central Toronto, Yorkville",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,...,0.00000,0.000000,0.000000,0.00,0.047619,0.000000,0.000000,0.000000,0.000000,0.00
7,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Church and Wellesley,0.012346,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.012346,0.000000,0.012346,0.00
9,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.026316,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']


for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cheese Shop,Steakhouse,Bakery,Farmers Market,Café,Beer Bar,Cocktail Bar,Department Store
1,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Skate Park,Smoke Shop,Spa,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Juice Bar,Burger Joint,Japanese Restaurant,Ice Cream Shop,Chinese Restaurant,Thai Restaurant,Bakery
3,"Central Toronto, Forest Hill West",Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,"Central Toronto, Summerhill East",Playground,Tennis Court,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


k = 2, 3 and 4 were tried for number of clusters. Due to "homogeneity" (clusters of 1-2 boroughs) 2 is selected

In [44]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

A dataframe with the data of the most common venues along with postal code information is created

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dft

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Park,Other Great Outdoors,Trail,Health Food Store,Asian Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner
1,M4K,East Toronto,"East Toronto, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Dessert Shop,Diner,Pub,Caribbean Restaurant
2,M4L,East Toronto,"East Toronto, India Bazaar",43.668999,-79.315572,1,Park,Pizza Place,Gym,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Italian Restaurant,Liquor Store,Movie Theater,Coffee Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Yoga Studio,Park,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Gym,Hotel,Dance Studio,Sandwich Place,Food & Drink Shop,Department Store,Breakfast Spot,Park,Dumpling Restaurant,Donut Shop
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Dessert Shop,Mexican Restaurant,Salon / Barbershop,Chinese Restaurant,Fast Food Restaurant,Sporting Goods Shop,Diner
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Toy / Game Store,Gourmet Shop
8,M4T,Central Toronto,"Central Toronto, Summerhill East",43.689574,-79.383160,1,Playground,Tennis Court,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,M4V,Central Toronto,"Central Toronto, Summerhill West",43.686412,-79.400049,1,Pub,Coffee Shop,Health & Beauty Service,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station


The map will show the different boroughs in Toronto along with its corresponding cluster

In [47]:
# create map
map_clusters = folium.Map(location=[43.7, -79.5], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Park,Other Great Outdoors,Trail,Health Food Store,Asian Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner
4,Central Toronto,0,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,Downtown Toronto,0,Park,Playground,Trail,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,Central Toronto,0,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Furniture / Home Store,Ice Cream Shop,Dessert Shop,Diner,Pub,Caribbean Restaurant
2,East Toronto,1,Park,Pizza Place,Gym,Sushi Restaurant,Ice Cream Shop,Fish & Chips Shop,Italian Restaurant,Liquor Store,Movie Theater,Coffee Shop
3,East Toronto,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Yoga Studio,Park,Seafood Restaurant
5,Central Toronto,1,Gym,Hotel,Dance Studio,Sandwich Place,Food & Drink Shop,Department Store,Breakfast Spot,Park,Dumpling Restaurant,Donut Shop
6,Central Toronto,1,Clothing Store,Coffee Shop,Yoga Studio,Dessert Shop,Mexican Restaurant,Salon / Barbershop,Chinese Restaurant,Fast Food Restaurant,Sporting Goods Shop,Diner
7,Central Toronto,1,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Toy / Game Store,Gourmet Shop
8,Central Toronto,1,Playground,Tennis Court,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,Central Toronto,1,Pub,Coffee Shop,Health & Beauty Service,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station
11,Downtown Toronto,1,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Pizza Place,Chinese Restaurant,Pub,Café,Park,Butcher
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gastropub


### Conclusion is not very significant, but it can be said that cluster 0 has park and women's store as the most popular venues (trail is also present in 3/4 of boroughs up to 4th place). Cluster 1 would just be the rest, Café and Coffee shop as most common venues could be a feature.